In [2]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)
library(splines)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘repr’ was built under R version 4.1.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“package ‘tidymodels’ was built under R version 4.1.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.2.0 ──

✔ broom        0.7.12     ✔ rsample      0.1.1 
✔ dials        0.1.0      ✔ tune         0.2.0 
✔ infer        1.0.0      ✔ workflows    0.2.6 
✔ modeldata   

In [9]:
setwd("/Users/matthewgillies")
metricdata <- read_csv("downloads/mlb-batter-exit-velocity.csv") %>% select(-id, -rank)
head(metricdata)

Rows: 1983 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): player
dbl (19): id, rank, year, batted_ball_events, launch_angle, sweet_spot_perce...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,player,batted_ball_events,launch_angle,sweet_spot_percentage,max_ev,average_ev,fly_ball_line_drive_ev,ground_ball_ev,max_distance,average_distance,average_homerun,hard_hit_95mph+,hard_hit_percentage,hard_hit_swing_percentage,total_barrels,barrels_batted_balls_percentage,barrels_plate_appearance_percentage
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022,"Judge, Aaron",341,14.6,37.8,118.4,95.8,100.2,89.1,465,205,413,210,61.6,20.3,91,26.7,15.8
2022,"Alvarez, Yordan",304,12.1,40.1,117.4,95.5,98.2,92.7,469,193,403,186,61.2,23.7,59,19.4,12.7
2022,"Trout, Mike",234,24.7,37.2,114.4,91.7,94.9,87.7,472,218,407,120,51.3,16.5,46,19.7,11.6
2022,"Schwarber, Kyle",312,18.9,34.6,114.8,93.3,99.7,87.2,468,197,415,170,54.5,17.6,64,20.5,11.5
2022,"Stanton, Giancarlo",228,10.2,26.3,119.8,94.6,98.2,94.1,445,160,400,117,51.3,17.8,42,18.4,11.3
2022,"Ohtani, Shohei",354,12.6,34.7,119.1,92.6,97.2,87.7,462,180,408,168,47.5,16.2,62,17.5,11.1


In [33]:
hr <- read_csv("downloads/stats (14).csv") %>% unite("player", last_name:first_name, sep = ", ", remove = T) %>%
select(-"...7") %>% rename("HR" = "b_home_run") %>% mutate("HR/AB" = HR/b_ab) %>% select(-HR, -b_ab)
head(hr)

New names:
• `` -> `...7`
Rows: 1112 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): last_name, first_name
dbl (4): player_id, year, b_ab, b_home_run
lgl (1): ...7

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


player,player_id,year,HR/AB
<chr>,<dbl>,<dbl>,<dbl>
"Cruz Jr., Nelson",443558,2022,0.02232143
"Blackmon, Charlie",453568,2022,0.03018868
"McCutchen, Andrew",457705,2022,0.03300971
"Turner, Justin",457759,2022,0.02777778
"Andrus, Elvis",462101,2022,0.03177570
"Santana, Carlos",467793,2022,0.04408353


In [34]:
alldata <- right_join(hr, metricdata, by = c("player", "year")) %>% select(-"player_id")
head(alldata)

player,year,HR/AB,batted_ball_events,launch_angle,sweet_spot_percentage,max_ev,average_ev,fly_ball_line_drive_ev,ground_ball_ev,max_distance,average_distance,average_homerun,hard_hit_95mph+,hard_hit_percentage,hard_hit_swing_percentage,total_barrels,barrels_batted_balls_percentage,barrels_plate_appearance_percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Blackmon, Charlie",2022,0.03018868,388,12.0,36.6,109.5,86.1,90.1,82.5,453,160,409,119,30.7,12.5,19,4.9,3.7
"McCutchen, Andrew",2022,0.03300971,353,13.6,34.6,110.6,89.2,93.1,85.7,428,168,390,141,39.9,15.6,27,7.6,5.5
"Turner, Justin",2022,0.02777778,331,19.0,40.2,108.9,89.4,91.8,86.6,414,191,392,133,40.2,15.7,27,8.2,6.0
"Andrus, Elvis",2022,0.03177570,351,12.7,28.8,109.9,87.9,90.7,88.0,430,153,383,117,33.3,14.4,13,3.7,2.8
"Santana, Carlos",2022,0.04408353,284,15.5,31.0,111.3,90.5,93.3,89.5,434,169,400,124,43.7,17.4,23,8.1,5.6
"Gurriel, Yuli",2022,0.01467890,400,14.6,30.8,107.1,88.2,90.6,89.1,401,161,377,141,35.3,16.2,8,2.0,1.6


In [35]:
write_csv(alldata, "/Users/matthewgillies/EVDATA")